¿QUE ES BOKEH?

Libreria de Python para crear visualizaciones interactivas basadas en navegador, y es adecuado para aplicaciones bigdata

In [ ]:
from bokeh.plotting import figure, show, save, output_file
from bokeh.layouts import gridplot, column, row
from bokeh.models import ColumnDataSource, HoverTool, Legend, LinearColorMapper, ColorBar
from bokeh.palettes import Spectral3, Viridis256
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.models import Spacer


In [ ]:
!pip install bokeh_sampledata
from bokeh.sampledata.autompg import autompg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 11.0 MB/s eta 0:00:00


Descripcion general del dataframe...

Contiene info sobre consumo de combustible y algunas especificaciones de carros vendidos en estados unidos entre los años 70 y 82

Al parecer es muy usado para aprender a usar la visualizacion de bokeh.

Info acerca de las columnas del dataframe:
*  mpg // millas por galon

*  cyl // numero de cilindros del motor

*  displ // cilindrada

*  hp   // caballos de fuerza (horsepower)

*  weight // peso de carro (lbs)

*  accel // tiempo de aceleracion

*  yr // año del modelo (ultimos digitos del año)

*  origin //  1 = EE. UU., 2 = Europa, 3 = Japón

*  name // marca y modelo del auto





In [ ]:
from bokeh.io import output_notebook
output_notebook()     ##representa el donde se realizara el output de la visualizacion de datos, es decir, mi ipynb

In [ ]:
import numpy as np
import pandas as pd
import warnings

In [ ]:
##load data
df = autompg.copy()
print(f"Dataset shape: {df.shape}")

Dataset shape: (392, 9)


In [ ]:
##limpiar datos
df = df.dropna()
print(f"Datos después de limpiar: {df.shape}")

Datos después de limpiar: (392, 9)


In [ ]:
##analisis estadistico basico
print(df[['mpg', 'hp', 'weight', 'displ']].describe())


              mpg          hp       weight       displ
count  392.000000  392.000000   392.000000  392.000000
mean    23.445918  104.469388  2977.584184  194.411990
std      7.805007   38.491160   849.402560  104.644004
min      9.000000   46.000000  1613.000000   68.000000
25%     17.000000   75.000000  2225.250000  105.000000
50%     22.750000   93.500000  2803.500000  151.000000
75%     29.000000  126.000000  3614.750000  275.750000
max     46.600000  230.000000  5140.000000  455.000000


In [ ]:
# asigne un mapa de color a cada pais, asumiendo que aplicare overplots
color_map = {
    1: '#e74c3c',  # USA - Rojo
    2: '#3498db',  # Europa - Azul
    3: '#2ecc71'   # Japón - Verde
}

In [ ]:
origin_names = {1: 'USA', 2: 'Europe', 3: 'Japan'}
df['origin_name'] = df['origin'].map(origin_names)
df['color'] = df['origin'].map(color_map)

En bokeh, hay algo llamado columndatasource, que es una estructura "principal" para almacenar y manejar datos que se graficaran. Sirve para actualizar dinamicamente los graficos cuando los datos cambian...


un linked brushing en bokeh es una tecnica de visualizacion de datos donde la seleccion de elementos en un grafico afecta automaticamente a otros valores

Esta tecnica es util a la hora de hacer un EDA para poder ver visualmente como un subconjunto de datos se comporta en distintas dimensiones simultaneamente



In [ ]:
 # ColumnDataSource compartido para aplicar Linked Brushing
source = ColumnDataSource(data=dict(
    mpg=df['mpg'],
    hp=df['hp'],
    weight=df['weight'],
    displ=df['displ'],
    yr=df['yr'],
    origin=df['origin'],
    origin_name=df['origin_name'],
    color=df['color'],
    name=df['name']
))

print(f"Registros en source: {len(source.data['mpg'])}")


Registros en source: 392


Hovertool


Se usa para visualizar info extra de todos los items de un grafico cuando el mouse para por encima de ellos.

Esto enriquece la visualizacion de datos sin saturar la grafica con un chorro de cosas

In [ ]:
## configuracion general de plots
#herramientas tipicas para la interactividad
TOOLS = "pan, box_zoom, save, wheel_zoom, reset, lasso_select, box_select"

#hoovertool
hover = HoverTool(tooltips=[
    ("Modelo", "@name"),
    ("MPG", "@mpg"),
    ("HP", "@hp"),
    ("Peso", "@weight lbs"),
    ("Año", "@yr"),
    ("Origen", "@origin_name")
])

PRIMER GRAFICO....

scatterplot


In [ ]:
#lienzo del scatter plot
p1 = figure(
    width=500, height=400,
    title="MPG vs Peso (Overplot: Puntos + Tendencia + Densidad)",
    x_axis_label="Peso (lbs)",
    y_axis_label="Millas por Galón (MPG)",
    tools=[hover] + TOOLS.split(',')
)
## CAPA 1: puntos de fondo grandes y transparentes (densidad)

density_scatter = p1.scatter(
    'weight', 'mpg', source=source,
    size=20, alpha=0.2, color='color',
    line_color=None,
    legend_label="Densidad"
)

## CAPA 2: línea de tendencia (es parecido a la regresion lineal)
z = np.polyfit(df['weight'], df['mpg'], 1)
p_trend = np.poly1d(z)
weight_trend = np.linspace(df['weight'].min(), df['weight'].max(), 100)
mpg_trend = p_trend(weight_trend)

trend_line = p1.line(
    weight_trend, mpg_trend,
    line_width=3, line_color='#9b59b6',
    line_dash='dashed', alpha=0.8,
    legend_label="Tendencia Linear"
)

# CAPA 3: puntos principales
main_scatter = p1.scatter(
    'weight', 'mpg', source=source,
    size=8, alpha=0.8, color='color',
    line_color='black', line_width=1,
    legend_label="Datos por Origen"
)
print(f"   - Correlación peso-MPG: r = {np.corrcoef(df['weight'], df['mpg'])[0,1]:.3f}")


   - Correlación peso-MPG: r = -0.832


SEGUNDO GRAFICO

Histograma

In [ ]:
  # lienzo del histograma
p2 = figure(
    width=500, height=400,
    title="Distribución MPG por Origen (Overplot de Histogramas)",
    x_axis_label="Millas por Galón (MPG)",
    y_axis_label="Frecuencia",
    tools=TOOLS
)

origins = [1, 2, 3]
colors = ['#e74c3c', '#3498db', '#2ecc71']
origin_labels = ['USA', 'Europe', 'Japan']

for i, (origin, color, label) in enumerate(zip(origins, colors, origin_labels)):
    origin_data = df[df['origin'] == origin]['mpg']

    # OVERPLOT: histogramas superpuestos con transparencia
    hist, edges = np.histogram(origin_data, bins=15, density=True)

    p2.quad(
        top=hist, bottom=0,
        left=edges[:-1], right=edges[1:],
        fill_color=color, line_color='white',
        alpha=0.6, legend_label=label
    )

    print(f"   - {label}: {len(origin_data)} vehículos, MPG promedio: {origin_data.mean():.1f}")

   - USA: 245 vehículos, MPG promedio: 20.0
   - Europe: 68 vehículos, MPG promedio: 27.6
   - Japan: 79 vehículos, MPG promedio: 30.5


GRAFICO 3

Series de tiempo

In [ ]:
#lienzo de la serie de tiempo
p3 = figure(
    width=1000, height=400,
    title="Evolución Temporal del MPG (Overplot: Puntos + Promedios + Rango)",
    x_axis_label="Año",
    y_axis_label="Millas por Galón (MPG)",
    tools=[hover] + TOOLS.split(',')
)

## CAPA 1: area de rango (fondo)
years = sorted(df['yr'].unique())
year_stats = df.groupby('yr')['mpg'].agg(['mean', 'min', 'max']).reset_index()

# Crear polígono para el rango
area_x = np.concatenate([year_stats['yr'], year_stats['yr'][::-1]])
area_y = np.concatenate([year_stats['max'], year_stats['min'][::-1]])

p3.patch(area_x, area_y, alpha=0.2, fill_color='gray',
         line_color=None, legend_label="Rango Min-Max")

# CAPA 2: línea de promedio anual
p3.line(year_stats['yr'], year_stats['mean'],
        line_width=4, line_color='#f39c12',
        alpha=0.9, legend_label="Promedio Anual")

# CAPA 3: puntos individuales
p3.scatter('yr', 'mpg', source=source,
          size=6, alpha=0.6, color='color',
          legend_label="Datos Individuales")

print(f"   - Mejora promedio MPG: {year_stats['mean'].iloc[-1] - year_stats['mean'].iloc[0]:.1f}")

   - Mejora promedio MPG: 14.3


  GRAFICO 4

  Heatmap

In [ ]:
##liennzo del heatmap

p4 = figure(
    width=500, height=400,
    title="HP vs Displacement (Overplot: Heatmap + Scatter)",
    x_axis_label="Caballos de Fuerza (HP)",
    y_axis_label="Cilindrada (cu in)",
    tools=[hover] + TOOLS.split(',')
)


# CAPA 1: color mapping por MPG (heatmap effect)
color_mapper = LinearColorMapper(palette=Viridis256,
                               low=df['mpg'].min(),
                               high=df['mpg'].max())

heatmap_scatter = p4.scatter(
    'hp', 'displ', source=source,
    size=12, alpha=0.8,
    color={'field': 'mpg', 'transform': color_mapper},
    line_color='black', line_width=1,
    legend_label="MPG (Color)"
)
## este modulo se usa para mostrar correspondencias entre valores de los datos con los colores de una visualizacion
color_bar = ColorBar(color_mapper=color_mapper, width=8, location=(0,0),
                    title="MPG")
p4.add_layout(color_bar, 'right')

# CAPA 2: contornos de densidad (simulado con círculos)
#  grid de densidad
hp_bins = np.linspace(df['hp'].min(), df['hp'].max(), 10)
displ_bins = np.linspace(df['displ'].min(), df['displ'].max(), 10)

# Crear listas para los datos de densidad
density_hp = []
density_displ = []
density_sizes = []

for hp_center in hp_bins[::2]:  # cada dos para evitar sobrecarga
    for displ_center in displ_bins[::2]:
        # puntos cercanos
        nearby = ((df['hp'] - hp_center)**2 + (df['displ'] - displ_center)**2) < 500
        density = nearby.sum()

        if density > 2:  # Solo visualizar donde hay densidad
            density_hp.append(hp_center)
            density_displ.append(displ_center)
            density_sizes.append(density*2)

# Ahora agregar los círculos de densidad usando los datos preparados
if density_hp:  # Solo si hay datos de densidad
    p4.scatter(density_hp, density_displ,
              size=density_sizes, alpha=0.1,
              color='red', line_color=None)



Ahora supuestamente ya estan creadas las graficas, falta configurar el linked brushing



In [ ]:
# esto determinara la posicion de cada grafica en el lienzo donde saldran todas juntas
for p in [p1, p2, p3, p4]:
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"

Visualizacion final...

In [ ]:
grid = column(
    row(p1, p2),
    p3,
    row(p4, Spacer(width=500, height=400))
)
show(grid)

**Inferencias de las graficas**

**Gráfico 1: MPG vs Peso (Scatter Plot con Overplot)t**

Relación negativa entre peso y consumo de combustible

Relación física clara: Los autos más pesados consumen más combustible (menor MPG)

**Gráfico 2: Distribución MPG por Origen (Histogramas Superpuestos)**

Japón lidera en eficiencia: Probablemente tiene el MPG promedio más alto

USA: Más vehículos pero posiblemente menor eficiencia promedio

Europa: Punto medio entre los otros dos paises


**Gráfico 3: Evolución Temporal del MPG (Serie de Tiempo)**

Progreso constante en eficiencia de combustible

**Gráfico 4: HP vs Displacement (Heatmap con Scatter)**

Más potencia/cilindrada = menor eficiencia (colores más oscuros en valores altos)


El análisis revela una industria automotriz en transición, donde factores externos (crisis energética, regulaciones) impulsaron innovaciones tecnológicas medibles. Las diferencias regionales muestran enfoques culturales distintos hacia el diseño automotriz, mientras que las correlaciones físicas (peso-eficiencia) demuestran principios universales de ingeniería.

**Conclusion**

Como estudiante nunca habia creado graficas tan elaboradas, bokeh a pesar de todos los problemas que tuve al aprender a usar, ya que no es como matplot que con seis lineas de codigo ya tienes tu grafica. Y el uso de las capas me saco corajes que no podria haberme esperado...

Esta herramienta de visualizacion realmente logra crear graficas tan detalladas sin sobrecargargar el grafico o hacerlo pesado de ver, tambien su uso dinamico apto para bigdata me hizo que me agradara mas.